In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with Grounding with Gemini in Vertex AI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/intro-grounding-gemini.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fgrounding%2Fintro-grounding-gemini.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/intro-grounding.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/intro-grounding-gemini.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

| | |
|-|-|
|Author(s) | [Holt Skinner](https://github.com/holtskinner), [Kristopher Overholt](https://github.com/koverholt) |

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.11

## Overview

[Grounding in Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/grounding/ground-language-models) lets you use generative text models to generate content grounded in your own documents and data. This capability lets the model access information at runtime that goes beyond its training data. By grounding model responses in Google Search results or data stores within [Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/enterprise-search-introduction), LLMs that are grounded in data can produce more accurate, up-to-date, and relevant responses.

Grounding provides the following benefits:

- Reduces model hallucinations (instances where the model generates content that isn't factual)
- Anchors model responses to specific information, documents, and data sources
- Enhances the trustworthiness, accuracy, and applicability of the generated content

In the context of grounding in Vertex AI, you can configure two different sources of grounding:

1. Google Search results for data that is publicly available and indexed
2. [Data stores in Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/create-datastore-ingest), which can include your own data in the form of website data, unstructured data, or structured data

**NOTE:** Some of the features in this sample notebook require early access to certain features via an allowlist. [Grounding with Vertex AI Search](https://cloud.google.com/vertex-ai/docs/generative-ai/grounding/ground-language-models) is available in Public Preview, whereas Grounding with Google Web Search results is available in Private Preview. To request early access to features in Private Preview, contact your account representative or [Google Cloud Support](https://cloud.google.com/contact).

### Objective

In this tutorial, you learn how to:

- Generate LLM text and chat model responses grounded in Google Search results
- Compare the results of ungrounded LLM responses with grounded LLM responses
- Create and use a data store in Vertex AI Search to ground responses in custom documents and data
- Generate LLM text and chat model responses grounded in Vertex AI Search results

This tutorial uses the following Google Cloud AI services and resources:

- Vertex AI
- Vertex AI Search and Conversation

The steps performed include:

- Configuring the LLM and prompt for various examples
- Sending example prompts to generative text and chat models in Vertex AI
- Setting up a data store in Vertex AI Search with your own data
- Sending example prompts with various levels of grounding (no grounding, web grounding, data store grounding)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.
1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).
1. Enable the [Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com) and [Vertex AI Search and Conversation API](https://console.cloud.google.com/flows/enableapi?apiid=discoveryengine.googleapis.com).
1. If you want to use Grounding with Google Web Search results, your project must also be allowlisted for this feature while it is in the Private Preview stage.
1. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

### Installation

Install the following packages required to execute this notebook.

In [ ]:
%pip install --upgrade --user --quiet google-cloud-aiplatform==1.42.1

Restart the kernel after installing packages:

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>

### Configure your project ID & region

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [15]:
PROJECT_ID = "your-project-id"  # @param {type:"string"}
REGION = "us-central1"  # @param {type: "string"}

# Set the project ID
!gcloud auth application-default set-quota-project {PROJECT_ID}
!gcloud config set project {PROJECT_ID}

### Authenticate your Google Cloud account

If you are running this notebook on Google Colab, you will need to authenticate your environment. To do this, run the new cell below. This step is not required if you are using Vertex AI Workbench.

In [ ]:
import sys

if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Import libraries

In [2]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Tool, grounding

from IPython.display import display, Markdown

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project:

In [16]:
vertexai.init(project=PROJECT_ID, location=REGION)

Initialize the generative text and chat models from Vertex AI:

In [17]:
model = GenerativeModel("gemini-1.0-pro")

## Example: Grounding with Google Search results

In this example, you'll compare LLM responses with no grounding with responses that are grounded in the results of a Google Search. You'll ask a question about a recent hardware release from the Google Store.

In [5]:
PROMPT = (
    "What are the price, available colors, and storage size options of a Pixel Tablet?"
)

### Text generation without grounding

Make a prediction request to the LLM with no grounding:

In [6]:
response = model.generate_content(PROMPT)

display(Markdown(response.candidates[0].text))

As of my knowledge cutoff in April 2023, the Pixel Tablet has not yet been officially announced or released. Therefore, details such as pricing, color options, and storage size options are not yet available. 

However, you can check the official Google website or reputable tech news sources for the latest and most up-to-date information on the Pixel Tablet once it becomes available.

### Text generation grounded in Google Search results

Now you can add the `tools` keyword arg with a grounding tool of `grounding.GoogleSearchRetrieval()` to instruct the LLM to first perform a Google Search with the prompt, then construct an answer based on the web search results:

In [7]:
tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

response = model.generate_content(PROMPT, tools=[tool])

display(Markdown(response.candidates[0].text))

response

- Price: Starting from USD 499
- Storage size options: 128 GB and 256 GB
- Available colors: Porcelain, Rose, and Hazel

candidates {
  content {
    role: "model"
    parts {
      text: "- Price: Starting from USD 499\n- Storage size options: 128 GB and 256 GB\n- Available colors: Porcelain, Rose, and Hazel"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
  grounding_metadata {
    web_search_queries: "Pixel Tablet price, colors, and storage size options?"
    grounding_attributions {
      web {
        uri: "https://store.google.com/product/pixel_tablet_specs"
        title: "Pixel Tablet Technical Specs - Google Store"
      }
      segment {
        end_index: 119
      }
      confidence_score: 0.778106511
    }
  }
}
usage_metadata {
  prompt_t

Note that the response without grounding only has limited information from the LLM about the Pixel tablet. Whereas the response that was grounded in web search results contains the most up to date information from web search results that are returned as part of the LLM with grounding request.

## Example: Grounding with custom documents and data

In this example, you'll compare LLM responses with no grounding with responses that are grounded in the [results of a data store in Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/create-datastore-ingest). You'll ask a question about a GoogleSQL query to create an [object table in BigQuery](https://cloud.google.com/bigquery/docs/object-table-introduction).

### Creating a data store in Vertex AI Search

Follow the steps in the [Vertex AI Search getting started documentation](https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#create_a_search_app_for_website_data) to create a data store in Vertex AI Search with sample data. In this example, you'll use a website-based data store that contains content from the Google Cloud website, including documentation.

Note: The data store must be in the same project that you are using for Gemini.

Once you've created a data store, obtain the Data Store ID and input it below.

In [7]:
DATA_STORE_PROJECT_ID = PROJECT_ID  # @param {type:"string"}
DATA_STORE_REGION = "global"  # @param {type:"string"}
# Replace this with your data store ID from Vertex AI Search
DATA_STORE_ID = "your-data-store-id_1234567890123"  # @param {type:"string"}

Now you can ask a question about object tables in BigQuery and when to use them:

In [6]:
PROMPT = "When should I use an object table in BigQuery? And how does it store data?"

### Text generation without grounding

Make a prediction request to the LLM with no grounding:

In [8]:
response = model.generate_content(PROMPT)

display(Markdown(response.candidates[0].text))

**When to Use an Object Table:**

Use an object table in BigQuery when you need to store and query structured data in a semi-nested format, particularly when:

* You have data with complex or hierarchical relationships.
* You want to efficiently query data across multiple levels of nesting.
* You need to represent parent-child relationships in your data.
* You want to model data that naturally fits into a hierarchical or tree-like structure.

**How Object Tables Store Data:**

Object tables store data as a collection of objects. Each object can have a unique set of fields and can be nested within other objects. The data structure is represented using a JSON-like format called **Arrow Flight Schema**.

An object table can have the following elements:

* **Root Object:** The top-level object in the table.
* **Nested Objects:** Objects within other objects, representing hierarchical relationships.
* **Primitive Fields:** Fields that store basic data types (e.g., string, number, date).
* **Complex Fields:** Fields that contain nested objects or arrays of objects.
* **Arrays:** Ordered lists of objects or primitive values.

**Example:**

Consider the following data representing employees and their departments:

```json
{
  "employees": [
    {
      "name": "Alice",
      "department": {
        "name": "Engineering",
        "manager": "Bob"
      }
    },
    {
      "name": "Bob",
      "department": {
        "name": "Management",
        "manager": "Carol"
      }
    }
  ]
}
```

This data can be represented in an object table as follows:

* **Root Object:** `employees` (an array of employee objects)
* **Nested Object:** `department` (an object representing the department for each employee)
* **Primitive Fields:** `name` (employee name), `manager` (department manager)
* **Complex Field:** `department` (contains nested objects)
* **Array:** `employees` (a collection of employee objects)

By using an object table, you can easily query data across different levels of nesting, such as finding all employees in the "Engineering" department or the manager of "Bob".

### Text generation grounded in Vertex AI Search results

Now we can add the `tools` keyword arg with a grounding tool of `grounding.VertexAISearch()` to instruct the LLM to first perform a search within your custom data store, then construct an answer based on the relevant documents:

In [9]:
datastore = f"projects/{DATA_STORE_PROJECT_ID}/locations/{DATA_STORE_REGION}/collections/default_collection/dataStores/{DATA_STORE_ID}"
tool = Tool.from_retrieval(
    grounding.Retrieval(grounding.VertexAISearch(datastore=datastore))
)

response = model.generate_content(PROMPT, tools=[tool])

display(Markdown(response.candidates[0].text))

response

Here are the scenarios when you should use an object table in BigQuery:

* **When you need to analyze unstructured data in Cloud Storage.** 
Object tables let you analyze unstructured data in Cloud Storage, such as images, videos, and audio files. 
* **When you need to perform analysis with remote functions.** 
Object tables let you perform analysis with remote functions, such as image classification and speech recognition. 
* **When you need to perform inference by using BigQuery ML.** 
Object tables let you perform inference by using BigQuery ML, such as predicting the sentiment of text or the genre of music.

BigQuery object tables store data in Cloud Storage. The data is stored in a columnar format, which makes it efficient for querying. Object tables also support partitioning, which can improve query performance.

candidates {
  content {
    role: "model"
    parts {
      text: "Here are the scenarios when you should use an object table in BigQuery:\n\n* **When you need to analyze unstructured data in Cloud Storage.** \nObject tables let you analyze unstructured data in Cloud Storage, such as images, videos, and audio files. \n* **When you need to perform analysis with remote functions.** \nObject tables let you perform analysis with remote functions, such as image classification and speech recognition. \n* **When you need to perform inference by using BigQuery ML.** \nObject tables let you perform inference by using BigQuery ML, such as predicting the sentiment of text or the genre of music.\n\nBigQuery object tables store data in Cloud Storage. The data is stored in a columnar format, which makes it efficient for querying. Object tables also support partitioning, which can improve query performance."
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
 

Note that the response without grounding only has limited information from the LLM about object tables in BigQuery that might not be accurate. Whereas the response that was grounded in Vertex AI Search results contains the most up to date information from the Google Cloud documentation about BigQuery, along with citations of the information.

## Example: Grounded chat responses

You can also use grounding when working with chat models in Vertex AI. In this example, you'll compare LLM responses with no grounding with responses that are grounded in the results of a Google Search and a data store in Vertex AI Search.

You'll ask a question about Vertex AI and a follow up question about managed datasets in Vertex AI:

In [10]:
PROMPT = "What are managed datasets in Vertex AI?"
PROMPT_FOLLOWUP = "What types of data can I use?"

### Chat session without grounding

Start a chat session and send messages to the LLM with no grounding:

In [12]:
chat = model.start_chat()

response = chat.send_message(PROMPT)
display(Markdown(response.text))

response = chat.send_message(PROMPT_FOLLOWUP)
display(Markdown(response.text))

Managed datasets in Vertex AI are scalable, high-performance datasets that can be easily accessed and managed by AI and machine learning models. These datasets are hosted on Google Cloud Storage and are automatically managed by Vertex AI. 

Managed datasets are designed to simplify the process of working with large and complex datasets. This allows data scientists and machine learning engineers to focus on building and training models, rather than managing the underlying infrastructure.

Managed datasets also provide a secure and scalable way to share datasets with collaborators. With managed datasets, users can grant access to specific users or groups, and can easily revoke access when needed. 

Overall, managed datasets are a valuable tool for data scientists and machine learning engineers who are looking to simplify the process of working with large and complex datasets.

You can use a wide variety of data types with managed datasets in Vertex AI, including:

* **Structured data:** This type of data is organized into rows and columns, and can be easily imported from CSV, JSON, or BigQuery files. Examples of structured data include customer data, sales data, and financial data.
* **Unstructured data:** This type of data is not organized into a specific format, and can include text, images, audio, and video. Examples of unstructured data include social media posts, product reviews, and medical records.
* **Semi-structured data:** This type of data has some structure, but is not as rigidly organized as structured data. Examples of semi-structured data include JSON and XML files.

Managed datasets also support a variety of data formats, including:

* CSV
* JSON
* BigQuery
* Apache Parquet
* Apache Avro

You can also use managed datasets to import data from Google Cloud Storage, BigQuery, and other data sources.

Once you have imported your data into a managed dataset, you can use it to train and evaluate machine learning models. Managed datasets are designed to be scalable and high-performance, so you can use them to train even the most complex models.

Here are some examples of how you can use managed datasets in Vertex AI:

* Train a machine learning model to predict customer churn using structured data from a CRM system.
* Train a machine learning model to classify images of products using unstructured data from a product catalog.
* Train a machine learning model to translate text from one language to another using semi-structured data from a translation corpus.

Managed datasets are a versatile and powerful tool that can be used to train and evaluate machine learning models on a wide variety of data types and formats.

### Chat session grounded in Google Search results

Now you can add the `tools` keyword arg with a grounding tool of `grounding.GoogleSearchRetrieval()` to instruct the chat model to first perform a Google Search with the prompt, then construct an answer based on the web search results:

In [20]:
chat = model.start_chat()
tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

response = chat.send_message(PROMPT, tools=[tool])
display(Markdown(response.text))
print(response)

response = chat.send_message(PROMPT_FOLLOWUP, tools=[tool])
display(Markdown(response.text))
print(response)

Managed datasets in Vertex AI are a fully managed data service that provides a central repository for your training and prediction data. It offers features like data versioning, schema management, and data quality monitoring. Managed datasets also simplify data sharing and collaboration within your organization.

candidates {
  content {
    role: "model"
    parts {
      text: "Managed datasets in Vertex AI are a fully managed data service that provides a central repository for your training and prediction data. It offers features like data versioning, schema management, and data quality monitoring. Managed datasets also simplify data sharing and collaboration within your organization."
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
  grounding_metadata {
    web_search_queries: "What are managed datasets in Vertex AI?"
  }
}
usage_metadata {
  prompt_token_count: 8
  candidates_token_count: 51
  total_token_count: 59
}



Managed datasets in Vertex AI support the following data types:

* **Tabular data:** Data in a tabular format, such as CSV or BigQuery tables.
* **Image data:** Images in JPEG, PNG, or GIF format.
* **Video data:** Videos in MP4 or MOV format.
* **Text data:** Text files in TXT or JSON format.

candidates {
  content {
    role: "model"
    parts {
      text: "Managed datasets in Vertex AI support the following data types:\n\n* **Tabular data:** Data in a tabular format, such as CSV or BigQuery tables.\n* **Image data:** Images in JPEG, PNG, or GIF format.\n* **Video data:** Videos in MP4 or MOV format.\n* **Text data:** Text files in TXT or JSON format."
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
  grounding_metadata {
    web_search_queries: "Vertex AI managed datasets"
  }
}
usage_metadata {
  prompt_token_count: 67
  candidates_token_count: 76
  total_token_count: 143
}



### Chat session grounded in Vertex AI Search results

Now we can add the `tools` keyword arg with a grounding tool of `grounding.VertexAISearch()` to instruct the chat model to first perform a search within your custom data store, then construct an answer based on the relevant documents:

In [21]:
chat = model.start_chat()
datastore = f"projects/{DATA_STORE_PROJECT_ID}/locations/{DATA_STORE_REGION}/collections/default_collection/dataStores/{DATA_STORE_ID}"
tool = Tool.from_retrieval(
    grounding.Retrieval(grounding.VertexAISearch(datastore=datastore))
)

response = chat.send_message(PROMPT, tools=[tool])
display(Markdown(response.text))
print(response)

response = chat.send_message(PROMPT_FOLLOWUP, tools=[tool])
display(Markdown(response.text))
print(response)

Managed datasets in Vertex AI are datasets that are stored and managed by Google Cloud. This means that you don't have to worry about the underlying infrastructure or the management of the data. You can simply access the data through the Vertex AI API or UI.

Managed datasets are a good option for those who want to focus on building and training models without having to worry about the data management. They are also a good option for those who need to share data with others in a secure and controlled way.

Here are some of the benefits of using managed datasets in Vertex AI:

* **Reduced operational overhead:** You don't have to worry about the underlying infrastructure or the management of the data.
* **Improved security:** Managed datasets are stored in a secure and controlled environment.
* **Increased collaboration:** You can easily share data with others in a secure and controlled way.
* **Simplified data management:** You can easily manage your data through the Vertex AI API or UI.

If you are interested in using managed datasets in Vertex AI, you can learn more here: https://cloud.google.com/vertex-ai/docs/datasets/overview

candidates {
  content {
    role: "model"
    parts {
      text: "Managed datasets in Vertex AI are datasets that are stored and managed by Google Cloud. This means that you don\'t have to worry about the underlying infrastructure or the management of the data. You can simply access the data through the Vertex AI API or UI.\n\nManaged datasets are a good option for those who want to focus on building and training models without having to worry about the data management. They are also a good option for those who need to share data with others in a secure and controlled way.\n\nHere are some of the benefits of using managed datasets in Vertex AI:\n\n* **Reduced operational overhead:** You don\'t have to worry about the underlying infrastructure or the management of the data.\n* **Improved security:** Managed datasets are stored in a secure and controlled environment.\n* **Increased collaboration:** You can easily share data with others in a secure and controlled way.\n* **Simplified da

With managed datasets in Vertex AI, you can use the following types of data:

* Tabular data
* Image data
* Video data
* Text data
* Time series data

You can also use managed datasets to store and manage data that is used for training and deploying models. This data can include:

* Training data
* Test data
* Validation data
* Model artifacts
* Predictions

Managed datasets are a good option for storing and managing data that is used for machine learning because they are:

* **Secure:** Data is stored in a secure and controlled environment.
* **Scalable:** Datasets can be scaled to accommodate large amounts of data.
* **Flexible:** Datasets can be used for a variety of machine learning tasks.

To learn more about managed datasets in Vertex AI, you can visit the following link: https://cloud.google.com/vertex-ai/docs/datasets/overview

candidates {
  content {
    role: "model"
    parts {
      text: "With managed datasets in Vertex AI, you can use the following types of data:\n\n* Tabular data\n* Image data\n* Video data\n* Text data\n* Time series data\n\nYou can also use managed datasets to store and manage data that is used for training and deploying models. This data can include:\n\n* Training data\n* Test data\n* Validation data\n* Model artifacts\n* Predictions\n\nManaged datasets are a good option for storing and managing data that is used for machine learning because they are:\n\n* **Secure:** Data is stored in a secure and controlled environment.\n* **Scalable:** Datasets can be scaled to accommodate large amounts of data.\n* **Flexible:** Datasets can be used for a variety of machine learning tasks.\n\nTo learn more about managed datasets in Vertex AI, you can visit the following link: https://cloud.google.com/vertex-ai/docs/datasets/overview"
    }
  }
  finish_reason: STOP
  safety_ratings {
    categor

## Cleaning up

To avoid incurring charges to your Google Cloud account for the resources used in this notebook, follow these steps:

1. To avoid unnecessary Google Cloud charges, use the [Google Cloud console](https://console.cloud.google.com/) to delete your project if you do not need it. Learn more in the Google Cloud documentation for [managing and deleting your project](https://cloud.google.com/resource-manager/docs/creating-managing-projects).
1. If you used an existing Google Cloud project, delete the resources you created to avoid incurring charges to your account. For more information, refer to the documentation to [Delete data from a data store in Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/delete-datastores), then delete your data store.
1. Disable the [Vertex AI Search and Conversation API](https://pantheon.corp.google.com/apis/api/discoveryengine.googleapis.com) and [Vertex AI API](https://pantheon.corp.google.com/apis/api/aiplatform.googleapis.com) in the Google Cloud Console.